In [1]:
import modern_robotics as mr
import numpy as np
import numpy.matlib
from calibration import *
numpy.set_printoptions(precision=16)

In [5]:
s = 0.001

w1 = np.array([0,0,1])
p1 = np.array([0,0,0])

w2 = np.array([0,-1,0])
p2 = np.array([0,0,0])

w3 = np.array([0,-1,0])
p3 = np.array([0,0,-100])

w4 = np.array([0,0,-1])
p4 = np.array([-50,250,0])*s

w5 = np.array([0,-1,0])
p5 = np.array([-20,0,-250])*s

w6 = np.array([0,0,-1])
p6 = np.array([-50,250,0])*s



s = 0.001
ax1 = mr.ScrewToAxis(p1,w1,0)
ax2 = mr.ScrewToAxis(p2,w2,0)
ax3 = mr.ScrewToAxis(p3,w3,0)
ax4 = mr.ScrewToAxis(p4,w4,0)
ax5 = mr.ScrewToAxis(p5,w5,0)
ax6 = mr.ScrewToAxis(p6,w6,0)


xi0 = np.c_[ax1, ax2, ax3, ax4, ax5, ax6]

wist0 = np.array([0,0,0])
pist0 = np.array([250,50,-20])*s

xist0 = mr.ScrewToAxis(wist0, pist0, 0)

xi01=np.array([0.0399999800000150,-0.0199999900000075,0.998999500500375,0.0200000000000000,0.0400000000000000,0])
xi02=np.array([0,-1,0,-0.0200000000000000,0,0.0500000000000000])
xi03=np.array([0.178005251232368,-0.984029029284552,-0.00100002950130544,-0.0841845888907446,0.0874136824072620,-100.999920311298])
xi04=np.array([0.0619994730067192,0.0129998895014089,-0.997991517108157,-50.9999969248523,249.000000644789,0.0751505000414993])
xi05=np.array([0.000999959501660306,-0.999999500040373,0,-20.6000000008239,-0.0205991760337826,-249])
xi06=np.array([0.0949994775043106,0.0309998295014066,-0.994994527545148,-51.2730269967030,248.910906980023,2.85959854441601])

xi00=np.c_[xi01,xi02,xi03,xi04,xi05,xi06]
xi00[3:5,:] = xi00[3:5,:]*s

#initial twist
xist00=np.array([0.02, -0.01, 0.01, 249, 51, -20.6])
xist00[3:6] = xist00[3:6]*s

deq1=0
deq2=0.02
deq3=0.002
deq4=0.02

N=10 

P01=np.array([100,0,0])*s
P02=np.array([0,100,0])*s
P03=np.array([0,0,100])*s
PX=np.c_[P01,P02,P03]

P04=np.array([-100,-100,-100])*s

mean_error_before = np.zeros((10,1))
mean_error_traditional = np.zeros((10,1))

max_error_before = np.zeros((10,1))
max_error_traditional = np.zeros((10,1))

In [11]:
gm=np.zeros((4,4,N))
gn=np.zeros((4,4,N))
Pa1=np.zeros((4,N))
Pa2=np.zeros((4,N))
Pa3=np.zeros((4,N))
M_home = np.eye(4)
cali_joint_angles = np.random.rand(N,6) * 2 * np.pi - np.pi


for i in range(0,N):
    
    thetalist = cali_joint_angles[i,:]

    Pa1[:,i] = np.dot(mr.FKinBody(M_home, xi00, thetalist), np.r_[P01,1].reshape((4,1)) ).reshape((4,)) + ((np.random.rand(4,1)*0.2-0.1) * s).reshape((4,))
    Pa2[:,i] = np.dot(mr.FKinBody(M_home, xi00, thetalist), np.r_[P02,1].reshape((4,1)) ).reshape((4,)) + ((np.random.rand(4,1)*0.2-0.1) * s).reshape((4,))
    Pa3[:,i] = np.dot(mr.FKinBody(M_home, xi00, thetalist), np.r_[P03,1].reshape((4,1)) ).reshape((4,)) + ((np.random.rand(4,1)*0.2-0.1) * s).reshape((4,))
    PY = np.c_[Pa1[0:3,i],Pa2[0:3,i],Pa3[0:3,i]]

    R,t = Registration(PX, PY)

    gm[0:3,0:3,i] = R
    gm[0:3,3,i] = t
    gn[:,:,i] = mr.FKinBody(M_home, xi00, thetalist)



In [15]:
def traditionalCalibrationPuma(xi0, xist0, vtheta, gm, M):

    xi = np.copy(xi0)

    xist = np.copy(xist0)

    M_home = np.eye(4)

    N = np.size(vtheta,0)

    gn = np.zeros((4,4,N))
    dg = np.zeros((4,4,N))
    vLog = np.zeros((6,N))

    for m in range(0,M):

        for i in range(0,N):

            gn[:,:,i]= mr.FKinBody(M_home, xi, vtheta[i,:])
            dg[:,:,i] = np.linalg.solve(gn[:,:,i].T, gm[:,:,i].T).T
            vLog[:,i]= vlog(dg[:,:,i])

        simY = np.zeros(6*N, 1)

        for i in range(1, N+1):

            simY[6*i - 5: 5*i, 0] = vLog[:,i]

        
        simJ = np.zeros((600,42))


        for k in range(0,N):

            mat1 = se3Exp( np.dot(vtheta[k,0], xi[:,0]))
            a_mat1 = aMatrix(xi[:,0], vtheta[k,0])

            mat2 = se3Exp( np.dot(vtheta[k,1], xi[:,1]))
            a_mat2 = aMatrix(xi[:,1], vtheta[k,1])

            mat3 = se3Exp( np.dot(vtheta[k,2], xi[:,2]))
            a_mat3 = aMatrix(xi[:,2], vtheta[k,2])

            mat4 = se3Exp( np.dot(vtheta[k,3], xi[:,3]))
            a_mat4 = aMatrix(xi[:,3], vtheta[k,3])

            mat5 = se3Exp( np.dot(vtheta[k,4], xi[:,4]))
            a_mat5 = aMatrix(xi[:,4], vtheta[k,4])

            mat6 = se3Exp( np.dot(vtheta[k,5], xi[:,5]))
            a_mat6 = aMatrix(xi[:,5], vtheta[k,5])

            a_mat_st = aMatrixST(xist)

            simJ[6*k:6+6*k,0:6  ] = a_mat1
            simJ[6*k:6+6*k,6:12 ] = np.dot(mr.Adjoint(mat1), a_mat2)
            simJ[6*k:6+6*k,12:18] = np.dot(mr.Adjoint(np.linalg.multi_dot[mat1, mat2]), a_mat3)
            simJ[6*k:6+6*k,18:24] = np.dot(mr.Adjoint(np.linalg.multi_dot[mat1, mat2, mat3]), a_mat4)
            simJ[6*k:6+6*k,24:30] = np.dot(mr.Adjoint(np.linalg.multi_dot[mat1, mat2, mat3, mat4]), a_mat5)
            simJ[6*k:6+6*k,30:36] = np.dot(mr.Adjoint(np.linalg.multi_dot[mat1, mat2, mat3, mat4, mat5]), a_mat6)
            simJ[6*k:6+6*k,36:42] = np.dot(mr.Adjoint(np.linalg.multi_dot[mat1, mat2, mat3, mat4, mat5, mat6]), a_mat_st)

        dp = np.linalg.lstsq(simJ, simY, rcond=None)[0]


        xi[:,0] = (xi[:,0].reshape((6,1))+dp[0:6].reshape((6,1))).reshape((6,))
        xi[0:3,0] = xi[0:3,0] / np.linalg.norm(xi[0:3,0])
        xi[3:6,0] = xi[3:6,0] + np.dot( np.dot(xi[0:3,1].T, xi[3:6,0]) /  np.dot(xi[0:3,0].T, xi[0:3,0] ), xi[0:3,0])

        xi[:,1] = (xi[:,1].reshape((6,1)) + dp[6:12].reshape((6,1))).reshape((6,))
        xi[0:3,1] = xi[0:3,1]/np.linalg.norm(xi[0:3,1])
        xi[3:6,1] = xi[3:6,1] - np.dot( np.dot(xi[0:3,1].T, xi[3:6,1]) / np.dot(xi[0:3,1].T,xi[0:3,1]), xi[0:3,1])

        xi[:,2] = (xi[:,2].reshape((6,1)) + dp[12:18].reshape((6,1))).reshape((6,))
        xi[0:3,2] = xi[0:3,2]/np.linalg.norm(xi[0:3,2])
        xi[3:6,2] = xi[3:6,2] - np.dot( np.dot(xi[0:3,2].T, xi[3:6,2]) / np.dot(xi[0:3,2].T,xi[0:3,2]), xi[0:3,2])

        xi[:,3]= (xi[:,3].reshape((6,1)) + dp[18:24].reshape((6,1))).reshape((6,))
        xi[0:3,3]=xi[0:3,3] / np.linalg.norm(xi[0:3,3])
        xi[3:6,3]= xi[3:6,3] - np.dot(np.dot(xi[0:3,3].T, xi[3:6,3]) / np.dot(xi[0:3,3].T,xi[0:3,3]),xi[0:3,3])

        xi[:,4]= (xi[:,4].reshape((6,1)) + dp[24:30].reshape((6,1))).reshape((6,))
        xi[0:3,4]=xi[0:3,4] / np.linalg.norm(xi[0:3,4])
        xi[3:6,4]= xi[3:6,4] - np.dot(np.dot(xi[0:3,4].T, xi[3:6,4]) / np.dot(xi[0:3,4].T,xi[0:3,4]),xi[0:3,4])

        xi[:,5]= (xi[:,5].reshape((6,1)) + dp[30:36].reshape((6,1))).reshape((6,))
        xi[0:3,5]=xi[0:3,5] / np.linalg.norm(xi[0:3,5])
        xi[3:6,5]= xi[3:6,5] - np.dot(np.dot(xi[0:3,5].T, xi[3:6,5]) / np.dot(xi[0:3,5].T,xi[0:3,5]),xi[0:3,5])

        vtheta[:,0] = vtheta[:,0]+dp[6]
        vtheta[:,1] = vtheta[:,1]+dp[13]
        vtheta[:,2] = vtheta[:,2]+dp[17]
        vtheta[:,3] = vtheta[:,3]+dp[24]

        xist = xist + dp[36:42]

    return xi, xist


In [ ]:

# calibration
xiTrad, dqTrad, meanETrad, convergenceTrad = traditionalCalibrationScara(xi0, vtheta, gm, 10)
Nt=50
error_before=np.zeros((Nt,1))
error_afterMinimal=np.zeros((Nt,1))
error_afterTraditional=np.zeros((Nt,1))
test_joint_config = np.c_[np.random.rand(Nt,1) * 2 * np.pi, np.random.rand(Nt,1) * 2 * np.pi, np.random.rand(Nt,1) * 1000 * s, np.random.rand(Nt,1) * 2 * np.pi ]


for i in range(0,Nt):
    
    error_before[i] = np.linalg.norm( np.dot( mr.FKinBody(M_home, np.round(xi0,4), test_joint_config[i,:]) - mr.FKinBody(M_home, np.round(xi00,4), test_joint_config[i,:] + np.array([deq1,deq2,deq3,deq4]) ), np.r_[P00,1])) / s
    
    error_afterTraditional[i] = np.linalg.norm( np.dot( mr.FKinBody(M_home, np.round(xiTrad,4), (test_joint_config[i,:] + dqTrad.T).reshape( (4,) ) ) - mr.FKinBody(M_home, np.round(xi00,4), test_joint_config[i,:] + np.array([deq1,deq2,deq3,deq4]) ), np.r_[P00,1]) ) / s


print("Mean error = {}".format(np.mean(error_before)))
print("Mean error calibrated = {}".format(np.mean(error_afterTraditional)))

print("Max error before = {}".format(np.max(error_before)))
print("Max error calibrated = {}".format(np.max(error_afterTraditional)))

array([[ 1.1377401466905592,  0.890844113494837 ,  1.3899682114758543,
        -2.226146798729956 , -0.7459482307301499,  2.5694263013615872],
       [ 1.1119109614232174, -0.7376861357166447, -2.591265090356597 ,
        -0.3314901779877824, -1.043365475881068 , -2.151897749441011 ]])